In [2]:
import re, sys, os
from pprint import pprint
from pathlib import Path
from operator import xor
from collections import defaultdict, Counter

patterns = [
    (b' %s ', bytes),
    (b' %s. ', bytes),
    (b' %s,', bytes),
    (b' %s.', bytes),
    (b' %s, ', bytes),
    (b'. %s ', bytes),
    (b'. %s ', bytes.title),
    (b', %s ', bytes),
    (b'.%s ', bytes),
    (b'.%s ', bytes.title),
    (b',%s ', bytes),
]

def test_rez(rez):
    ss = rez.split()  # .lower()
    for s in ss:
        m = re.match(rb'^([A-Z]?[a-z]+-?[a-z]*)|(\d+[-.,]?\d*)[.,]?$', s)
#        m = re.match(rb'^[-A-Za-z0-9.,]+$', s)
        if not m:
            return
    return True

In [3]:
def hexToBytes(s):
    bs = []
    for i in range(0, len(s), 2):
        scod = s[i: i + 2]
        bs.append(int(scod, 16))
    return  bytes(bs)

# Зашифрований текст, перетворений на байтові рядки
txt1 = Path("shifrovka.txt").read_text()
lines = [hexToBytes(s.strip()) for s in txt1.splitlines() if s.strip()]
# lines = dict(enumerate(lines))
print(len(lines))

txt2 = Path("en_words.txt").read_text()
words = [bytes(s.strip(), encoding='utf8') for s in txt2.splitlines() if s.strip()]
words = sorted(words, key=len, reverse=True) 
print(len(words))

19
1013


In [4]:
# сортовані по спаду довжині рядка
lines2 = sorted(lines, reverse=True, key=len)
# несортовані, порядок як у початковій шифровці
# lines2 = lines

In [5]:
# Якщо кожний рядок зашифрований спільним ключем (послідовністю байтів),
# спробуємо знайти можливі значення і їх статистику для кожного байту
rez_codes = {}
for word0 in words:
    for ps in patterns:
        word = ps[1](ps[0] % word0)
        # print(word)
        # зсувом слова word для XOR кожної пари рядків шукаємо правдоподібний текст
        # якщо він правдоподібний для попередніх і наступних рядків, то скоріш за все
        # слово правильне і треба добавити статистику для цих позицій
        # (можна не тільки сусідніх, а в будьяких, тільки це накладніше)
        for L in range(1, len(lines2) - 1):
            # результат побайтного XOR для поточного і поаереднього рядків
            xorL0 = bytes(map(xor, lines2[L], lines2[L-1]))
            # результат побайтного XOR для поточного і наступного рядків
            xorL2 = bytes(map(xor, lines2[L], lines2[L+1]))
            # готовимо позицію зсувів
            for i in range(min(len(xorL0), len(xorL2)) - len(word) + 1):
                # одержуемо результати з зсунутими word-ами
                rez0 = bytes(map(xor, word, xorL0[i:]))
                rez2 = bytes(map(xor, word, xorL2[i:]))
                # перевіряємо правдоподібність результатів
                if test_rez(rez0) and test_rez(rez2):
                    # результати правдоподібні, добавляемо статистику
                    icodes = bytes(map(xor, word, lines2[L][i:]))
                    for j, cod in enumerate(icodes):
                        rez_codes.setdefault(i + j, Counter())[cod] += 1
rez_codes = {n: kvs.most_common(1)[0][0] for n, kvs in rez_codes.items()}
def_code = 0x42
print('''\
позиції 001111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789''')
for line in lines:
    rez = []
    for i, b in enumerate(line):
        ch = b ^ rez_codes.get(i, def_code)
        rez.append(ch)
    print(bytes(rez).decode(encoding='utf8', errors='replace'))

позиції 001111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789
jOr woo would bear tde whips and scorns .*lx,!<)
xH'opwressor's wrong  the proud man's co/89a   )
xHe pfngs of dispriz+d love, the law's d$ -ui
xHe iisolence of offece, and the spurns
xHat watient merit oj th'unworthy takes,
{Hen oe himself mighx his quietus make
{Ith f bare bodkin? [ho would fardels be >`
xO grrnt and sweat ubder a weary life,
nUt toat the dread oj something after de 8$ 
xHe uidiscovere'd coyntry, from whose bo4>"
bO trfveller returns  puzzles the will,
mNd mfkes us rather near those ills we h :)
xHan aly to others tdat we know not of?
xHus donscience doth,make cowards of us     
mNd tous the native due of resolution
eS sidklied o'er witd the pale cast of t)#9k-8u
mNd eiterprises of g~eat pith and moment
{Ith shis regard theer currents turn awr8
mNd lhse the name of,action.


In [7]:
from copy import deepcopy
from functools import reduce 

rez2_codes = deepcopy(rez_codes)

# Позиції, яких необхідно замінити символи
manuals = {
  5: b'oh', 
 20: b"dh", 
  0: b'ma', 
  1: b'Nn', 
 40: b'n/', 
 41: b't8', 
 42: b'u9', 
 43: b'ma', 
 44: b'e ', 
 45: b'l ', 
 46: b'y ', 
 47: b',k', 
}
# Зміна кодів для заміни символів в позиціях
for p, x in manuals.items():
    rez2_codes[p] = rez2_codes.get(p, def_code) ^ reduce(xor, x, 0)

for line in lines:
    rez = []
    for i, b in enumerate(line):
        ch = b ^ rez2_codes[i]#.get(i, def_code)
        rez.append(ch)
    print(bytes(rez).decode(encoding='utf8', errors='replace'))

for who would bear the whips and scorns of timen
th'oppressor's wrong, the proud man's contumelyn
the pangs of dispriz'd love, the law's delay,
the insolence of office, and the spurns
that patient merit of th'unworthy takes,
when he himself might his quietus make
with a bare bodkin? Who would fardels bear,
to grunt and sweat under a weary life,
but that the dread of something after death,
the undiscovere'd country, from whose bourn
no traveller returns, puzzles the will,
and makes us rather bear those ills we have
than fly to others that we know not of?
thus conscience doth make cowards of us all,
and thus the native hue of resolution
is sicklied o'er with the pale cast of thought,
and enterprises of great pith and moment
with this regard their currents turn awry
and lose the name of action.
